<a href="https://colab.research.google.com/github/clive-limo/DataScienceProjects/blob/main/Text_Completion_Prediction_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Next Word Prediction Model

In [ ]:
import numpy as np
from nltk.tokenize import RegexpTokenizer
from keras.models import Sequential, load_model
from keras.layers import LSTM
from keras.layers.core import Dense, Activation
from keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import pickle
import heapq

## Import Data

In [ ]:
text_data = open('1661-0.txt').read().lower()
len(text_data)

581888

In [ ]:
text_data = open('tv_text.txt').read().lower()
len(text_data)

11758510

# Data Preprocessing

In [ ]:
tokenizer = RegexpTokenizer(r'\w+')
words = tokenizer.tokenize(text_data)

In [ ]:
unique_words = np.unique(words)
unique_word_index = dict((c, i) for i, c in enumerate(unique_words))

### Feature Selection

In [ ]:
LENGTH = 5
previous = []
next = []
for i in range(len(words) - LENGTH):
    previous.append(words[i:i + LENGTH])
    next.append(words[i + LENGTH])
print(previous[0])
print(next[0])

['project', 'gutenberg', 's', 'the', 'adventures']
of


In [ ]:
X = np.zeros((len(previous), LENGTH, len(unique_words)), dtype=bool)
Y = np.zeros((len(next), len(unique_words)), dtype=bool)

In [ ]:
for i, each_words in enumerate(previous):
    for j, each_word in enumerate(each_words):
        X[i, j, unique_word_index[each_word]] = 1
    Y[i, unique_word_index[next[i]]] = 1

## Model

In [ ]:
model = Sequential()
model.add(LSTM(128, input_shape=(LENGTH, len(unique_words))))
model.add(Dense(len(unique_words)))
model.add(Activation('softmax'))

In [ ]:
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X, Y, validation_split=0.05, batch_size=128, epochs=20, shuffle=True).history

Epoch 1/20


/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


811/811 [==============================] - 268s 327ms/step - loss: 6.0057 - accuracy: 0.1081 - val_loss: 7.0514 - val_accuracy: 0.1022
Epoch 2/20
811/811 [==============================] - 271s 334ms/step - loss: 5.7764 - accuracy: 0.1469 - val_loss: 7.8327 - val_accuracy: 0.1018
Epoch 3/20
811/811 [==============================] - 263s 324ms/step - loss: 5.7378 - accuracy: 0.1753 - val_loss: 7.9695 - val_accuracy: 0.0961
Epoch 4/20
811/811 [==============================] - 264s 325ms/step - loss: 5.4120 - accuracy: 0.2093 - val_loss: 8.1755 - val_accuracy: 0.1011
Epoch 5/20
811/811 [==============================] - 261s 322ms/step - loss: 5.1160 - accuracy: 0.2491 - val_loss: 8.5880 - val_accuracy: 0.0967
Epoch 6/20
811/811 [==============================] - 261s 322ms/step - loss: 4.8402 - accuracy: 0.2868 - val_loss: 8.5174 - val_accuracy: 0.0959
Epoch 7/20
811/811 [==============================] - 266s 328ms/step - loss: 4.6010 - accuracy: 0.3239 - val_loss: 8.8040 - val_accura

In [ ]:
def format_text(text):
    x = np.zeros((1, LENGTH, len(unique_words)))
    for t, word in enumerate(text.split()):
        print(word)
        x[0, t, unique_word_index[word]] = 1
    return x

def sample(preds, top_n=3):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    return heapq.nlargest(top_n, range(len(preds)), preds.take)

def predict_next_words(text, n=3):
    if text == "":
        return("0")
    x = format_text(text)
    preds = model.predict(x, verbose=0)[0]
    next_indices = sample(preds, n)
    return [unique_words[idx] for idx in next_indices]

q =  "This place is"
print("Current sentence: ",q)
seq = " ".join(tokenizer.tokenize(q.lower())[0:5])
print("next possible words: ", predict_next_words(seq, 5))

Current sentence:  This place is
this
place
is
next possible words:  ['rift', 'fleecy', 'beautiful', 'waterproof', 'grain']
